In [ ]:
pip install generative-ai-hub-sdk

In [ ]:
pip install panda

In [ ]:
pip install hana-ml

In [ ]:
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
import os

os.environ["AICORE_AUTH_URL"] = "AI_CORE_URL"
os.environ["AICORE_CLIENT_ID"] = 'CLIENT_ID'
os.environ["AICORE_CLIENT_SECRET"] = 'SECRET'
os.environ["AICORE_RESOURCE_GROUP"] = 'USER_GROUP (USUAL IS DEFAULT)'
os.environ["AICORE_BASE_URL"] = "URL"


proxy_client = get_proxy_client('gen-ai-hub')

In [ ]:
from hana_ml import ConnectionContext
#cc = Connectioncontext(userkey='VDB_beta' encript=true)

cc = ConnectionContext(
    address='*********.hanacloud.ondemand.com',
    port=443, 
    user='DBUSER', 
    password='DBPASS', 
    encrypt=True
)

#print(cc.hana.version())
print(cc.get_current_schema())

In [ ]:

import hdbcli
from hdbcli import dbapi

conn = dbapi.connect(
    address='*********.hanacloud.ondemand.com',
    port=443, 
    user='DBUSER', 
    password='DBPASS', 
    encrypt=True
)
cursor = conn.cursor()

In [ ]:
pip install langchain

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from langchain.chains import RetrievalQA #ok
from langchain.prompts import PromptTemplate #ok


# Load PDF
loaders = [
     PyPDFLoader("PATH_TOPDF.pdf")]
docs = []
for loader in loaders:
    docs.extend(loader.load())
    
pdfdata=loader.load()
document = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2500, chunk_overlap=100)
text_chunks = text_splitter.split_documents(docs)
text_chunks[1]

In [ ]:
from gen_ai_hub.proxy.langchain.init_models import init_embedding_model 
from langchain_community.vectorstores.hanavector import HanaDB

embeddings = init_embedding_model('text-embedding-ada-002')
db = HanaDB(embedding=embeddings, connection=conn, table_name="PDF")
db.delete(filter={})
db.add_documents(text_chunks)
print(text_chunks)


In [42]:
#Get Embeddings

from gen_ai_hub.proxy.native.openai import embeddings
def get_embedding(input, model="text-embedding-ada-002") -> str: 
    response = embeddings.create(
    model_name = model,
    input=input
    )
    return response.data[0].embedding

# Setting page title and header
st.set_page_config(page_title="FAQ chatbot", page_icon=":speech-balloon:", initial_sidebar_state="collapsed")
st.title("FAQ chatbot")

# Sidebar
st.sidebar.title("Administration")
restart_button = st.sidebar.button("Start again", key="restart")

#Performing the cosine similarity search by pass the query vector.

import requests
import json 

def run_vector_search(query: str, metric="COSINE_SIMILARITY", k=4):
    if metric == 'L2DISTANCE': 
        sort = 'ASC'
    else: 
        sort ='DESC'
    query_vector = get_embedding(query)
    sql = '''SELECT TOP 2 "VEC_TEXT" FROM "DBADMIN"."PDF"
    ORDER BY "COSINE_SIMILARITY"("VEC_VECTOR", TO_REAL_VECTOR('{qv}')) {sort}'''.format(k=4,metric="COSINE_SIMILARITY",qv=query_vector,sort=sort)
    hdf = cc.sql(sql)
    df_context = hdf.head(k).collect()
    return df_context



In [45]:
query = "what models can I use?"
df_context = run_vector_search(query=query,metric="COSINE_SIMILARITY",k=4)
df_context

,VEC_TEXT
0,5. Enter the name and version (if applicable) ...
1,"2. In the ML Ops app, choose Scenarios and c..."
